# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [1]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing, libmpMuelMat, visualization_lines, plot_polarimetry

In [12]:
# set the parameters to run the script
directories = [r'E:\TumorMeasurements\dev']
calib_directory = r'E:\calib'

# set the parameters to be used for the line visualisation with the parameter parameter_set
parameter_set = 'TheoniPics'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
run_all = False

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes with PDDN when available and also without using the PDDN
PDDN_mode = 'both'

In [ ]:
batch_processing.batch_process_master(directories, calib_directory, run_all = run_all, 
                               parameter_set = parameter_set, PDDN = PDDN_mode)

1. processing without PDDN...


0it [00:00, ?it/s]


processing without PDDN done.

2. processing with PDDN.


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

No PDDN file found - denoising the raw data
 >> PDDN Loading Model: C:\Users\romai\Documents\repositories\processingMM\src\processingmm\PDDN_model\PDDN_model_550_Fresh_HB.pt -- ...
 [wrn] PDDN Model Summary NOT Available!
 >> PDDN Loading Model: C:\Users\romai\Documents\repositories\processingMM\src\processingmm\PDDN_model\PDDN_model_550_Fresh_HB.pt -- Complete
*** PDDN DENOISING FILTER *** 
 >> PDDN Denoising: total time-points: 1000
 >> PDDN Denoising: start time-point: 1
 >> PDDN Denoising: final time-point: 0
 >> PDDN Denoising: recursive filtering enabled (2 x cascade): False
 >> MPI Denoising PDDN Performance: Elapsed time = 3.188 s
 
 >> Exported X3D .cod file as: ./temp_processing\2023-06-20_T_HORAO-105-BF_FR_S1_1\raw_data\550nm\550_Intensite_PDDN.cod
